In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from facenet_pytorch import MTCNN
import torch


c:\Users\koushika\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Paths
image_dir = r"C:/Users/koushika/Dropbox/PC/Downloads/FERPlus-master/FERPlus_Balanced_5Class"
output_file = r"C:/Users/koushika/Dropbox/PC/Downloads/FERPlus-master/FERPlus_Balanced_5Class_landmarks.npy"

# MTCNN
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(keep_all=False, device=device)


In [3]:
landmarks_dict = {}

for split in ['train', 'test']:
    split_dir = os.path.join(image_dir, split)
    for cls in os.listdir(split_dir):
        cls_dir = os.path.join(split_dir, cls)
        for img_name in tqdm(os.listdir(cls_dir), desc=f"{split}/{cls}"):
            img_path = os.path.join(cls_dir, img_name)
            img_rel_path = f"{split}/{cls}/{img_name}"
            
            img = cv2.imread(img_path)
            if img is None:
                print(f"⚠️ Failed to load {img_path}")
                continue
            
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            _, _, landmarks = mtcnn.detect(img_rgb, landmarks=True)
            
            if landmarks is not None and len(landmarks) > 0:
                landmarks_array = landmarks[0]
            else:
                landmarks_array = np.zeros((5, 2), dtype=np.float32)
            
            landmarks_dict[img_rel_path] = landmarks_array


test/surprise: 100%|██████████| 700/700 [00:11<00:00, 61.37it/s]


In [4]:
np.save(output_file, landmarks_dict)
print(f"✅ Landmarks saved to {output_file}")


✅ Landmarks saved to C:/Users/koushika/Dropbox/PC/Downloads/FERPlus-master/FERPlus_Balanced_5Class_landmarks.npy


In [5]:
loaded_landmarks = np.load(output_file, allow_pickle=True).item()
print(f"Loaded {len(loaded_landmarks)} landmark entries")
list(loaded_landmarks.items())[:3]  # Show some sample entries


Loaded 19911 landmark entries


[('train/anger/fer0000010.png',
  array([[15.434951782226562, 20.939085006713867],
         [38.264564514160156, 21.710905075073242],
         [25.785295486450195, 30.056964874267578],
         [15.296342849731445, 44.3617057800293],
         [35.39720153808594, 45.454063415527344]], dtype=object)),
 ('train/anger/fer0000010.png.png',
  array([[15.434951782226562, 20.939085006713867],
         [38.264564514160156, 21.710905075073242],
         [25.785295486450195, 30.056964874267578],
         [15.296342849731445, 44.3617057800293],
         [35.39720153808594, 45.454063415527344]], dtype=object)),
 ('train/anger/fer0000022.png.png',
  array([[12.01125431060791, 16.053373336791992],
         [28.313356399536133, 15.09048843383789],
         [21.4791259765625, 25.831350326538086],
         [14.927038192749023, 34.25493240356445],
         [27.176713943481445, 33.38087463378906]], dtype=object))]